In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

too old and weird to be useful


Here is a list of the parameters that are planned to be read in from command line or so.
global_prefix -> for the output path and maybe the run.
Config_filename (maybe one for hoc, one for pseudo_hoc)
Input filename for the GC points
Input filename for the GoC coordinates



To do (still):
Check the dendrite directions
Check the generate somata randomly function or take it out (which would be sad though...

Write tests
Save tests




In [7]:
import sklearn
sklearn.__version_

AttributeError: module 'sklearn' has no attribute '__version_'

In [2]:
%run BREPpy.py

np.random.seed(0)

import pdb

p1 = './example_simulation/coordinates_input/'
p2 = './example_simulation/coordinates_input/subsampled/'

go_ori = p1+'GoCcoordinates.dat'
gr_ori = p1+'GCcoordinates.dat'
go_64 = p2+'GoCcoordinates_64.dat'
gr_64 = p2+'GCcoordinates_64.dat'
go_16 = p2+'GoCcoordinates_16.dat'
gr_16 = p2+'GCcoordinates_16.dat'
go_4 = p2+'GoCcoordinates_4.dat'
gr_4 = p2+'GCcoordinates_4.dat'

class Pseudo_hoc (object):
    '''Up to now, the program depends on a hoc object that contains the parameters for the simulation.
    However, as in the cluster, neuron is not installed for python 3, this class is a workaround:
    First, a dict has to be generated (and probably pickled) from the hoc file in a python distribution that has neuron installed.
    This dict (or the file containing it) is then read in in this class, and an empty object with no other functionalities gets
    assigned all the parameters from the dict as attributes. The resulting object can then be used as a parameter carrier just as the hoc file.'''
    
    def __init__ (self, ad_or_fn = []):
        '''Add parameters from dict or filename to pseudo_hoc object'''
        #Make a pseudo-hoc object
        if ad_or_fn == []: return
        elif type(ad_or_fn) == str:
            try:
                import pickle
                with open (ad_or_fn, 'rb') as f_in:
                    ad_or_fn = pickle.load(f_in)
            except: print ('Tried to read in ', ad_or_fn, ' as a file, failed')
        else: assert type(ad_or_fn) == dict, 'Could not read in ' + ad_or_fn
        # Add all elements from the read in file as arguments to the pseudo-hoc object
        for k,v in ad_or_fn.items():
            #As for the pickling process, all values had to be declared as strings, try to convert them back to a number
            try: v = float(v) 
            except: pass
            try: setattr(self, k, v)
            except: pass
            
    def convert_hoc_to_pickle (self, config_fn, output_fn = 'pseudo_hoc.pkl'):
        '''Take a .hoc config file and pickle it as a neuron-independent python dict.'''
        try: import neuron
        except: 
            print ('Could not import neuron, go to a python environment with an installed neuron version and try again.')
            return
        neuron.h.xopen(config_fn)
        d = dir(h)
        h_dict = dict()
        #Transfer parameters from the hoc object to a python dictionary
        for n,el in enumerate(d):
            #note! so far all used parameters in the BREPpy started with capital letters, so only adding atrributes that start with capital letters is a reasonable filtering method.
            #However, this is to be kept in mind when adding additional parameters to the parameter file.
            if el[0].isupper():
                try:
                    #The value has to be converted to its string representation to get rid of the hoc properties.
                    #Must be kept in mind when reading in though.
                    h_dict[el] = repr(getattr(h,el))
                except: pass
        #Dump the dictionary
        import pickle
        with open(output_fn, 'wb') as f:
            pickle.dump(h_dict, f)

            
try:
    notthiscell()
    import neuron
    config_fn = './input_files/Parameters.hoc'
    neuron.h.xopen(config_fn)
    print ('hoc object succesfully read in from ', config_fn)
except:
    config_fn = 'pseudo_hoc.pkl'
    h = Pseudo_hoc(config_fn)
    print ('pseudo-hoc object succesfully read in from ', config_fn)





pseudo-hoc object succesfully read in from  pseudo_hoc.pkl


In [3]:
ph = Pseudo_hoc()
ph.convert_hoc_to_pickle('./input_files/Parameters.hoc',  'pseudo_hoc2.pkl' )


In [5]:
h = Pseudo_hoc('pseudo_hoc2.pkl')

In [10]:
gg = Golgi_pop(h)
gg.load_somata(np.zeros((1,3)))
gg.add_dendrites()
gp = Granule_pop(h)
gp.load_somata(np.zeros((1,3)))
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()

(1, 24)
(1, 24, 2)


TypeError: object of type 'int' has no len()

In [6]:
# Set up the Golgi population, render dendrites
gg = Golgi_pop(h)
gg.load_somata(go_64)
gg.add_dendrites()


#Set up Granule population including aa and pf
gp = Granule_pop(h)
gp.load_somata(gr_64)
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()

#Build connector and obtain the connections
cc = Connect_2D(gg.qpts, gp.qpts_aa, 15)
res_aa, l_res_aa = cc.find_connections()
cc.save_results (res_aa, l_res_aa)
#cc.save_results(l1,r1, 'first_test/pts_in_lin_aa_')
#cc.save_results(l2,r2, 'first_test/lin_in_pts_aa_', query_is_lin=True)

cc = Connect_2D(gg.qpts, gp.qpts_pf, 15)
res_pf, l_res_pf = cc.find_connections()


#l21,r21 = cc.query_pts_in_lin()
#l22,r22 = cc.query_lin_in_pts()
#cc.save_results(l1,r1, 'first_test/pts_in_lin_pf')
#cc.save_results(l2,r2, 'first_test/lin_in_pts_pf', query_is_lin=True)

#gg.save_dend_coords ('first_test/')
#gg.save_somata ('first_test/', 'GoCcoordinates.dat')

#gp.save_gct_points('first_test/')
#gp.save_somata('first_test/', 'GCcoordinates.dat')


In [ ]:
bh = dict()
d = 'bla'
type (bh)
type(d) ==  str

In [ ]:
res_pf3= res_pf.copy()
res_aa3 = res_aa.copy()

In [ ]:
print (len(res_pf))
print (len(res_aa))

In [ ]:
print (np.all(np.array([np.all(res_pf1[i] == res_pf3[i]) for i in range(len(res_pf1))])))
print (np.all(np.array([np.all(res_aa1[i] == res_aa3[i]) for i in range(len(res_aa1))])))

In [ ]:
#v = var(h)


#print (len(v))
print (len(d))

class alibi (object):
    def __init__ (self, arg_dict):
        for k,v in arg_dict.items():
            try: setattr(self, k, v)
            except: pass

alibi_args = alibi(h_dict)




In [ ]:
h_dict.items()

In [ ]:
alibi_args2

In [ ]:
#turn the hoc file into a dict


config_fn = './input_files/Parameters.hoc'
import neuron
neuron.h.xopen(config_fn)
d = dir(h)


h_dict = dict()

for n,el in enumerate(d):
    if el[0].isupper():
        try:
            h_dict[el] = repr(getattr(h,el))
            #setattr(alibi_args, el, getattr(h,el))       
            #print (el)
        except: pass

        
import pickle
with open('pseudo_hoc.pkl', 'wb') as f:
    pickle.dump(h_dict, f)

In [ ]:
ar = np.load('pseudo')

In [ ]:
print (sum([len(l11[i]) for i in range(len(l11))]))
print (sum([len(l12[i]) for i in range(len(l12))]))
print (sum([len(l13[i]) for i in range(len(l13))]))

def ravel_list (l):
    return [i for ar in l for i in ar]
    





In [ ]:
from BREPpy import *
p1 = './example_simulation/coordinates_input/'
p2 = './example_simulation/coordinates_input/subsampled/'

go_ori = p1+'GoCcoordinates.dat'
gr_ori = p1+'GCcoordinates.dat'
go_64 = p2+'GoCcoordinates_64.dat'
gr_64 = p2+'GCcoordinates_64.dat'
go_16 = p2+'GoCcoordinates_16.dat'
gr_16 = p2+'GCcoordinates_16.dat'
go_4 = p2+'GoCcoordinates_4.dat'
gr_4 = p2+'GCcoordinates_4.dat' 


empty_hoc = dir(neuron.hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
neuron.h.xopen(config_fn)

#pdb.set_trace()


# Set up the Golgi population, render dendrites
gg = Golgi_pop(h)
gg.load_somata(go_64)
gg.add_dendrites()


#Set up Granule population including aa and pf
gp = Granule_pop(h)
gp.load_somata(gr_64)
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()

#Build connector and obtain the connections
cc = Connect_2D(gg.qpts, gp.qpts_aa, 15)
res_aa, l_res_aa = cc.find_connections()
cc.save_results (res_aa, l_res_aa)
#cc.save_results(l1,r1, 'first_test/pts_in_lin_aa_')
#cc.save_results(l2,r2, 'first_test/lin_in_pts_aa_', query_is_lin=True)

cc = Connect_2D(gg.qpts, gp.qpts_pf, 15)
res_pf, l_res_pf = cc.find_connections()




In [ ]:
l1_r = ravel_list(r11)
idx,nc= np.unique(r11_r, return_counts = True)

In [ ]:
cc.save_results(l2,r2, 'first_test/lin_in_pts_pf', query_is_lin=True)

In [ ]:
a = True
print (type(a))
print (type([]).__name__ == 'bool')

In [ ]:
print (type(gp).__name__)

In [ ]:
so = gg.qpts.seg

print (sg.shape)

dist_info = False

titles = ['AA', 'PF']

fc = so[:,0]
sc = so[:,1]
print ('First')
for i in np.unique(fc):
    idx = np.isclose(fc, i)
    if dist_info: print (i, ': n =', sum(idx), ', corresponding mean distance:', np.mean(dis_f[n][idx]))
    else: print (i, ': n =', sum(idx))
print ('Second')
for i in np.unique(sc):
    idx = np.isclose(sc, i)
    if dist_info: print (i, ': n =', sum(idx), ', corresponding mean distance:', np.mean(dis_f[n][idx]))
    else: print (i, ': n =', sum(idx))

In [ ]:
%run BREPpy.py
gg = Golgi_pop(h)
gg.load_somata(go_64)
gg.add_dendrites()
gg.save_dend_coords('teeeeest_')

In [ ]:
%run BREPpy.py

gp = Granule_pop(h)
gp.load_somata(gr_64)
gp.save_gct_points('teeeeest_')
gp.add_aa_endpoints_fixed()
gp.add_pf_endpoints()
gp.save_gct_points('teeeeest_')


In [ ]:
ad = gg.a_dend
print (ad.shape)
nn = ad[1]
#print (nn)
print (ad.ravel().shape)

In [ ]:
seg2 = seg.copy()

In [ ]:
t = np.array([1,2])
print (str(t))

In [ ]:
warnings.warn?


In [ ]:
r1, lr1 = cc.query_pts_in_lin()

In [ ]:
% run BREPpy.py
cc = Connect_2D(gg.qpts, gp.qpts_aa, 15)
cc.save_results(r1, lr1, 'test_AA')


In [ ]:
print (gp.aa_dots.shape)

print (gp.aa_dots[:2, :, :])
print (gp.aa_dots[:2, -1, :])

 

In [ ]:
#%run BREPpy.py
plt.figure(figsize = (12,8))

gg.plot_somata(False, 'ro')
gp.plot_somata(False, 'k.', markersize = 0.5)
plt.title ('Somata in the granule layer (Granule: black, Golgi: red)');

In [ ]:
## plot some full cells

#Different populations
gr_pop = gp
go_pop = gg

m = 0
#print (gg.qpts.idx[m])

#Get all Granule cells that are attached to one Golgi cell point (r1 comes from query_pts_in_lin)
go_cells = gg.qpts.idx[m]   #indices for the Golgi population
gr_cells =r1[m] #indices for the Granule cell population
q_pt =gg.qpts.coo[m]     # special (query?) points


# 3D plot
plt.figure()
ax = plt.gcf().gca(projection='3d')
def plot3d (ax, dat, *args, **kwargs):
    if dat == []: return
    if len(dat.shape) == 1: #if single dot
        dat = np.expand_dims(dat, axis = 0)
    ax.plot(dat[:,0], dat[:,1], dat[:,2], *args, **kwargs)

for c in gr_cells:
    plot3d(ax, gr_pop.aa_dots[c], 'c')
    plot3d(ax, gr_pop.pf_dots[c], 'b')
    plot3d(ax, gr_pop.som[c], 'bo')

for c in go_cells:
    plot3d(ax, go_pop.a_dend[c], 'rx')
    plot3d(ax, go_pop.b_dend[c], 'mx')
    plot3d(ax, np.expand_dims(go_pop.som[c], axis = 0), 'ro')

    plot3d(ax, q_pt, 'gx', markersize = 2)

    plt.title('3D space')
    
#2D projections
plt.figure(figsize=(15,5))
for i, [j, k, tit] in enumerate([[0,1, 'x-y plane'], [1,2, 'y-z plane'], [0,2, 'x-z plane']]):
    
    plt.subplot(1,3,i+1)
    
    for c in gr_cells:
        plt.plot(gr_pop.aa_dots[c,:,j], gr_pop.aa_dots[c,:,k], 'c')
        plt.plot(gr_pop.pf_dots[c,:,j], gr_pop.pf_dots[c,:,k], 'b')
        plt.plot(gr_pop.som[c,j], gr_pop.som[c,k], 'bo')

    for c in go_cells:
        plt.plot(go_pop.a_dend[c,:,j], go_pop.a_dend[c,:,k], 'rx')
        plt.plot(go_pop.b_dend[c,:,j], go_pop.b_dend[c,:,k], 'mx')
        plt.plot(go_pop.som[c,j], go_pop.som[c,k], 'ro')
        
    if len(q_pt) > 0: plt.plot(q_pt[j], q_pt[k], 'gx', markersize = 20)
    
    
    if True:
        if i in [0,2]: plt.xlim([-100, 500])
    plt.title (tit)
    
    
    plt.axis('equal')

In [ ]:
## plot some full cells
import matplotlib.pyplot as plt
%matplotlib inline

gr_pop = gp
go_pop = gg

a = np.random.randint(0, len(r2[0]), 5)
#ex = r2[a]


gr_cells = r2[0] #np.random.randint(0, len(gr_pop.som), 10)
go_cells = [0] #np.random.randint(0, len(go_pop.som), 2)


# 3D plot
plt.figure()
ax = plt.gcf().gca(projection='3d')
def plot3d (ax, dat, *args, **kwargs):
    if len(dat.shape) == 1: #if single dot
        dat = np.expand_dims(dat, axis = 0)
    ax.plot(dat[:,0], dat[:,1], dat[:,2], *args, **kwargs)

for c in gr_cells:
    plot3d(ax, gr_pop.aa_dots[c], 'c')
    plot3d(ax, gr_pop.pf_dots[c], 'b')
    plot3d(ax, gr_pop.som[c], 'bo')

for c in go_cells:
    plot3d(ax, go_pop.a_dend[c], 'r')
    plot3d(ax, go_pop.b_dend[c], 'm')
    plot3d(ax, np.expand_dims(go_pop.som[c], axis = 0), 'ro')
    
    
#2D projections
plt.figure(figsize=(15,5))
for i, [j, k, tit] in enumerate([[0,1, 'x-y plane'], [1,2, 'y-z plane'], [0,2, 'x-z plane']]):
    
    plt.subplot(1,3,i+1)
    
    for c in gr_cells:
        plt.plot(gr_pop.aa_dots[c,:,j], gr_pop.aa_dots[c,:,k], 'c')
        plt.plot(gr_pop.pf_dots[c,:,j], gr_pop.pf_dots[c,:,k], 'b')
        plt.plot(gr_pop.som[c,j], gr_pop.som[c,k], 'bo')

    for c in go_cells:
        plt.plot(go_pop.a_dend[c,:,j], go_pop.a_dend[c,:,k], 'r')
        plt.plot(go_pop.b_dend[c,:,j], go_pop.b_dend[c,:,k], 'm')
        plt.plot(go_pop.som[c,j], go_pop.som[c,k], 'ro')
    
    if True:
        if i in [0,2]: plt.xlim([-100, 500])
    plt.title (tit)

In [ ]:
qpt = Query_points(gg.som)

qpt.npts

In [ ]:
from neuron import hoc, h

%run BREPpy.py


empty_hoc = dir(neuron.hoc.HocObject()).copy()
config_fn = './input_files/Parameters.hoc'
#load our own hoc object from the parameter file, get the disjunct list of parameters (probably useless...)
neuron.h.xopen(config_fn)
gp = Granule_pop(h)
gp.load_somata('./example_simulation/coordinates_input/GCcoordinates.dat')


plt.figure()
ax = plt.gcf().gca(projection='3d')
ax.plot(gp.som[:,0], gp.som[:,1], gp.som[:,2], *args)

In [ ]:
from neuron import hoc, h

%run BREPpy.py
gp = Granule_pop('./example_simulation/coordinates_input/GCcoordinates.dat', h)
gp.add_aa_endpoints_random()



In [ ]:
    def read_in_config2 (self):
       '''checks if a config file has been specified, and if so, updates the args.
        If overwrite_config is set True, command line arguments that specify a parameter that also exists
        in the config file will have priority.'''
        if self.args.config_fn == '':
            warnings.warn('Cannot find config file!')
        else:  #read in config file using neurons h object
            self.p_verb('Reading config file')
            empty_hoc = dir(hoc.HocObject()).copy()
            neuron.h.xopen(self.args.config_fn)
            full_hoc = dir(neuron.h) 
        self.


In [ ]:
%run BREPpy.py

# Visualization of Golgi cells

For both the apical and the basal dendrites, a parameter GoC_*d_nseg is defined by `int((GoC_ApicalDendL/(GoC_d_lambda*GoC_d_lambda_f)+0.9)/2)*2 + 1 ` (analogous for BasolateralDendL)
Plugging in the values for the other parameters specified, I get that only if the length parameter GoC_**DendL is >= 488.6 (which seems an unlikely case), this parameter is not 1. (Values in the current config file are 166 for apical, 60 for basolateral)
I would like to understand what this parameter does and what it is used for (something cable equationish?)


# Visualization of Parallel fiber

# Visualization of Somata

# Single connection visualization

# Connection density visualization

In [ ]:
l = 166
fr = l/(0.1*1000*np.sqrt (3/(4*np.pi)))
print (fr)
nsegp = fr+0.9
nseg = int(nsegp/2)*2+1
print (nsegp)
print (nseg)


In [ ]:
int (0.9)

In [ ]:
b = 'ga'
print (type(b) == str)

In [ ]:
import threading
from queue import Queue
import time




In [ ]:

q

In [ ]:
%connect_info #-> gives the number and the address of the kernel you use
%qtconsole #-> opens the console with iqdb
%debug #-> starts a iqdb pocess inline
#-> here, if you print debug, you will jump to the last error. Leave with q

# pdb.set_trace() -> 

In [ ]:
# about the nseg parameter in the Parameters.hoc file.
l = 166
fr = l/(0.1*1000*np.sqrt (3/(4*np.pi)))
print (fr)
nsegp = fr+0.9
nseg = int(nsegp/2)*2+1
print (nsegp)
print (nseg)


In [ ]:
% run BREPpy
gol = './example_simulation/coordinates_input/subsampled/GoCcoordinates_64.dat'
h = Pseudo_hoc('pseudo_hoc.pkl')

gg = Golgi_pop(h)
gg.load_somata(gol)
#gg.gen_random_cell_loc(1995, 1500, 700, 200)
gg.add_dendrites()
gg.add_axon()
cc = Connect_3D(gg.axon_q, gg.b_dend_q, 30, './output_pybrep/spam/GoCtoGoC_')
_ cc.connections_parallel(True)